In [1]:
import json
import utilfuncs
import dataloader

cache_path = "Data/cache.json"

In [2]:
_ = utilfuncs.demo(cache_path, "Dra")

Papers included that have data on Dra:
['Kirby+ 2010', 'Reichert+ 2020', 'Duggan+, 2018']

Number of stars in Dra: 303

Columns included in Dra:
['logeps_Mg', 'e_tot_Ni', 'logeps_Cr', 'e_Fe_H', 'logeps_Sc', 'e_tot_Zn', 'logeps_Mn', 'e_a_Fe', 'logeps_Ni', 'logeps_Zn', 'Simbad', 'logeps_Eu', 'Imag', 'e_tot_Sr', 'Abs_abun', 'logeps_Sr', 'Element', 'logeps_Ba', 'Mg_Fe', 'Ba_Fe', 'e_log_g', 'T_eff', 'Fe_H', 'Absorp_feature_lambda', 'DEJ2000', 'log_g', 'K10', 'e_tot_Cr', 'RAJ2000', 'e_T_eff', 'a_Fe', 'Ion_state', 'e_tot_Eu', 'e_Ba_Fe', 'Ca_Fe', 'e_tot_Mn', 'logeps_Ti', 'e_tot_Sc', 'e_tot_Ba', 'e_tot_Ti', 'Vmag', 'e_tot_Mg', 'Galaxy', 'Si_Fe', 'Name', 'vt', 'Ti_Fe']
